In [1]:
import os
import cv2
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
class ImageClassifier():
    
    def __init__(self, data_path,im_size):
        self.data_path = data_path
        self.names = os.listdir(self.data_path)
        
        if '.DS_Store' in self.names:
            self.names.remove('.DS_Store')
            
        self.n_ims=0
        for name in self.names:
            self.n_ims += len(os.listdir(self.data_path+name))

        self.im_size = im_size
        self.get_data()
        self.compile_model()
    
    def get_image(self, filename):
        img = cv2.imread(filename)
        if img is None:
            return np.zeros((self.im_size, self.im_size, 3))
        else:
            img = cv2.resize(img, dsize=(self.im_size, self.im_size),interpolation=cv2.INTER_AREA)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            return img / 255
        
    def get_data(self):
        self.X = np.empty((self.n_ims, self.im_size, self.im_size, 3))
        self.y = np.empty(self.n_ims).T
        i = -1
        j = 0
        for name in self.names:
            directory = self.data_path + name
            for filename in os.listdir(directory):
                f = os.path.join(directory, filename)
                if os.path.isfile(f):
                    self.X[i, :, :, :] = self.get_image(f)
                    self.y[i] = j
                    i += 1
            j += 1
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.07)
        
    def compile_model(self):
        self.model = tf.keras.models.Sequential([
        # Convolution layers
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', input_shape=(self.im_size, self.im_size, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)), 
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        # Classification layers
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(self.names), activation='softmax')
        ])
        
        
        self.model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
        
    def fit_model(self, epochs):
        self.model.fit(self.X_train, self.y_train, epochs = epochs)
        self.acc = self.model.evaluate(self.X_test,self.y_test, batch_size=100)
        self.pred = np.argmax(self.model.predict(self.X_test), axis=1)
        print('test accuracy: {}'.format(self.acc[1]))

     
    def plot_predictions(self):
        plt.figure(figsize=(15, 15))
        for i in range(49):
            plt.subplot(7,7,i+1)
            plt.xticks([])
            plt.yticks([])
            plt.grid(False)
            plt.imshow(self.X_test[i])

            if self.y_test[i] != self.pred[i]:
                plt.title('Incorrect prediction', c='red', fontsize=15)

            else:
                plt.title(self.names[self.pred[i]],fontsize=20)

        plt.tight_layout()
        plt.show()
        

In [3]:
path = '/Users/ollie/Local_Documents/Uni_Work/Year_3/Statistical_Learning/Data/galaxy_dataset/'

In [4]:
clf = ImageClassifier(path, im_size=69)
clf.fit_model(20)
clf.plot_predictions()

2022-04-21 17:21:16.478747: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-21 17:21:16.478940: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1
Epoch 1/20


2022-04-21 17:21:18.487665: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/Users/ollie/miniforge3/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2022-04-21 17:21:18.679800: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 76/634 [==>...........................] - ETA: 51s - loss: 1.7443 - accuracy: 0.3059

KeyboardInterrupt: 

In [1]:
import cv2.cv2 as cv2
import numpy as np
 
from utils.image_classifier import ImageClassifier, NO_FACE_LABEL
 
# Color RGB Codes & Font
WHITE_COLOR = (255, 255, 255)
GREEN_COLOR = (0, 255, 0)
BLUE_COLOR = (255, 255, 104)
FONT = cv2.QT_FONT_NORMAL
 
# Frame Width & Height
FRAME_WIDTH = 640
FRAME_HEIGHT = 490
 
 
class BoundingBox:
    def __init__(self, x, y, w, h):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
 
    @property
    def origin(self) -> tuple:
        return self.x, self.y
 
    @property
    def top_right(self) -> int:
        return self.x + self.w
 
    @property
    def bottom_left(self) -> int:
        return self.y + self.h
 
 
def draw_face_rectangle(bb: BoundingBox, img, color=BLUE_COLOR):
    cv2.rectangle(img, bb.origin, (bb.top_right, bb.bottom_left), color, 2)
 
 
def draw_landmark_points(points: np.ndarray, img, color=WHITE_COLOR):
    if points is None:
        return None
    for (x, y) in points:
        cv2.circle(img, (x, y), 1, color, -1)
 
 
def write_label(x: int, y: int, label: str, img, color=BLUE_COLOR):
    if label == NO_FACE_LABEL:
        cv2.putText(img, label.upper(), (int(FRAME_WIDTH / 2), int(FRAME_HEIGHT / 2)), FONT, 1, color, 2, cv2.LINE_AA)
    cv2.putText(img, label, (x + 10, y - 10), FONT, 1, color, 2, cv2.LINE_AA)
 
 
class RealTimeEmotionDetector:
    CLAHE = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
 
    vidCapture = None
 
    def __init__(self, classifier_model: ImageClassifier):
        self.__init_video_capture(camera_idx=0, frame_w=FRAME_WIDTH, frame_h=FRAME_HEIGHT)
        self.classifier = classifier_model
 
    def __init_video_capture(self, camera_idx: int, frame_w: int, frame_h: int):
        self.vidCapture = cv2.VideoCapture(camera_idx)
        self.vidCapture.set(cv2.CAP_PROP_FRAME_WIDTH, frame_w)
        self.vidCapture.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_h)
 
    def read_frame(self) -> np.ndarray:
        rect, frame = self.vidCapture.read()
        return frame
 
    def transform_img(self, img: np.ndarray) -> np.ndarray:
        # load the input image, resize it, and convert it to gray-scale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # convert to gray-scale
        resized_img = self.CLAHE.apply(gray_img)  # resize
        return resized_img
 
    def execute(self, wait_key_delay=33, quit_key='q', frame_period_s=0.75):
        frame_cnt = 0
        predicted_labels = ''
        old_txt = None
        rectangles = [(0, 0, 0, 0)]
        landmark_points_list = [[(0, 0)]]
        while cv2.waitKey(delay=wait_key_delay) != ord(quit_key):
            frame_cnt += 1
 
            frame = self.read_frame()
            if frame_cnt % (frame_period_s * 100) == 0:
                frame_cnt = 0
                predicted_labels = self.classifier.classify(img=self.transform_img(img=frame))
                rectangles = self.classifier.extract_face_rectangle(img=frame)
                landmark_points_list = self.classifier.extract_landmark_points(img=frame)
            for lbl, rectangle, lm_points in zip(predicted_labels, rectangles, landmark_points_list):
                draw_face_rectangle(BoundingBox(*rectangle), frame)
                draw_landmark_points(points=lm_points, img=frame)
                write_label(rectangle[0], rectangle[1], label=lbl, img=frame)
 
                if old_txt != predicted_labels:
                    print('[INFO] Predicted Labels:', predicted_labels)
                    old_txt = predicted_labels
 
            cv2.imshow('Emotion Detection - Mimics', frame)
 
        cv2.destroyAllWindows()
        self.vidCapture.release()
 
 
def run_real_time_emotion_detector(
        classifier_algorithm: str,
        predictor_path: str,
        dataset_csv: str,
        dataset_images_dir: str = None):
    from utils.data_land_marker import LandMarker
    from utils.image_classifier import ImageClassifier
    from os.path import isfile
 
    land_marker = LandMarker(landmark_predictor_path=predictor_path)
 
    if not isfile(dataset_csv):  # If data-set not built before.
        print('[INFO]', f'Dataset file: "{dataset_csv}" could not found.')
        from data_preparer import run_data_preparer
        run_data_preparer(land_marker, dataset_images_dir, dataset_csv)
    else:
        print('[INFO]', f'Dataset file: "{dataset_csv}" found.')
 
    classifier = ImageClassifier(csv_path=dataset_csv, algorithm=classifier_algorithm, land_marker=land_marker)
    print('[INFO] Opening camera, press "q" to exit..')
    RealTimeEmotionDetector(classifier_model=classifier).execute()
 
 
if __name__ == "__main__":
    """The value of the parameters can change depending on the case."""
    run_real_time_emotion_detector(
        classifier_algorithm='RandomForest',  # Alternatively 'SVM'.
        predictor_path='utils/shape_predictor_68_face_landmarks.dat',
        dataset_csv='data/csv/dataset.csv',
        dataset_images_dir='data/raw'
    )
    print('Successfully terminated.')

ModuleNotFoundError: No module named 'utils'